# NSCM 32


Tero Frondelius, Thomas Helfer, Ivan Yashchuk, Joona Vaara, and Anssi Laukkanen


1. University of Oulu, firstname.lastname@oulu.fi
2. VTT Technical Research Centre of Finland, firstname.lastname@vtt.fi
3. Wärtsilä R&D and Engineering, firstname.lastname@wartsila.com 
4. CEA, DEN, DEC, France, thomas.helfer@cea.fr 

In [ ]:
norton = raw"""
@DSL Implicit;
@Author Thomas Helfer;
@Date 3 / 08 / 2018;
@Behaviour Norton;
@Description {
  "This file implements the Norton law "
  "using the StandardElastoViscoplasticity brick"
}

@ModellingHypotheses{".+"};
@Epsilon 1.e-16;

@Brick StandardElastoViscoPlasticity{
  stress_potential : "Hooke" {young_modulus : 150e6, poisson_ratio : 0.3},
  inelastic_flow : "Norton" {criterion : "Mises", A : 8.e-67, n : 8.2, K : 1}
};
""";

In [ ]:
mfront_fn = "norton.mfront"
open(mfront_fn,"w") do fil
    write(fil,norton)
end

In [ ]:
cur_dir = pwd()
pkg_dir = dirname(Base.find_package("MFrontInterface"))
lib_dir = joinpath(pkg_dir,"..","deps","usr","lib")
home_dir = joinpath(pkg_dir,"..","deps","usr")
bin_dir = joinpath(pkg_dir,"..","deps","usr","bin")
cur_path = ENV["PATH"]

In [ ]:
envfil = """
#!/bin/bash
export TFELHOME=$home_dir
export MGISHOME=$home_dir
export LD_LIBRARY_PATH=$lib_dir
export PATH=$bin_dir:$cur_path
mfront --install-path=$cur_dir --obuild --interface=generic $mfront_fn
patchelf --set-rpath $lib_dir src/libBehaviour.so
"""
open("build.sh","w") do fil
    write(fil,envfil)
end

In [ ]:
chmod("build.sh",0o777)
run(`./build.sh`)

In [ ]:
using MFrontInterface, FEMMaterials, Plots
mgis_bv = MFrontInterface.behaviour

In [ ]:
function MFrontMaterialFunction()
    lib_path = "src/libBehaviour.so"
    behaviour_name = "Norton"
    hypothesis = mgis_bv.Tridimensional

    behaviour = load(lib_path, behaviour_name, hypothesis)
    behaviour_data = BehaviourData(behaviour)

    ext_variable_names = [mgis_bv.get_name(mgis_bv.get_external_state_variables(behaviour)[i]) for i in 1:mgis_bv.length(mgis_bv.get_external_state_variables(behaviour))]
    ext_variable_values = zeros(length(ext_variable_names))
    ext_vatiable_state = MFrontExternalVariableState(names=ext_variable_names, values=ext_variable_values)

    return MFrontMaterial(behaviour=behaviour, behaviour_data=behaviour_data, external_variables=ext_vatiable_state)
end

In [ ]:
e11 = 0.0001*vcat(Array(range(0, stop=1.5e-2, length=30)), Array(range(1.5e-2, stop=-1.5e-2, length=30)), Array(range(-1.5e-2, stop=3e-2, length=40)))
e22 = -e11/2
e33 = e22
strains = [[e11[i], e22[i], e33[i], 0.0, 0.0, 0.0] for i in 1:100]

In [ ]:
plot([e11,e22,e33], labels=["e11","e22","e33"], xlabel="Time step", ylabel="Strain", grid=true)

In [ ]:
mat = MFrontMaterialFunction()
sim = Simulator(mat)
ts = 0.01:0.01:1.0
FEMMaterials.initialize!(sim, strains, ts)
FEMMaterials.run!(sim)

In [ ]:
s11 = []; str11 = []
for (stre, stra) in zip(sim.stresses,sim.strains)
    push!(s11,stre[1])
    push!(str11,stra[1])
end

In [ ]:
plot(ts,[[sim.stresses[i][j] for i=1:length(sim.stresses)] for j=1:3],xlabel="Time", ylabel="Stresses",labels=["S11","S22","S33"])

In [ ]:
download("https://raw.githubusercontent.com/JuliaFEM/FEMMaterials.jl/master/examples/data_3dbeam/plastic_beam.inp", "plastic_beam.inp")

In [ ]:
using JuliaFEM, Materials, FEMBase;
import FEMMaterials: Continuum3D, MecaMatSo

In [ ]:
mesh = abaqus_read_mesh("plastic_beam.inp");
beam_elements = create_elements(mesh, "Body1");
bc_elements_1 = create_nodal_elements(mesh, "BC1");
bc_elements_2 = create_nodal_elements(mesh, "BC2");
trac_elements = create_surface_elements(mesh, "PRESSURE");

In [ ]:
for j in 1:3
    update!(bc_elements_1, "displacement $j", 0.0)
end
update!(bc_elements_2, "displacement 1", 0.0)
update!(bc_elements_2, "displacement 2", 0.0)
update!(trac_elements, "surface pressure", 0.0 => 0.00)
update!(trac_elements, "surface pressure", 1.0 => 2.70e-4)
trac = Problem(Elasticity, "traction", 3)
bc = Problem(Dirichlet, "fix displacement", 3, "displacement")
add_elements!(trac, trac_elements)
add_elements!(bc, bc_elements_1)
add_elements!(bc, bc_elements_2)

In [ ]:
# These are IdealPlastic (Materials.jl) model parameters
update!(beam_elements, "youngs_modulus", 200.0e3)
update!(beam_elements, "poissons_ratio", 0.3)
update!(beam_elements, "yield_stress", 100.0)

In [ ]:
beam = Problem(Continuum3D, "plastic beam", 3)
beam.properties.material_model = :(Main.MFrontMaterialFunction) #:IdealPlastic #
add_elements!(beam, beam_elements)

In [ ]:
analysis = Analysis(MecaMatSo, "solve problem")
analysis.properties.max_iterations = 50
analysis.properties.t0 = 0.0
analysis.properties.t1 = 1.0
analysis.properties.dt = 0.05
temperature = 293.15
update!(beam_elements, "external_variables", [temperature])

In [ ]:
xdmf = Xdmf("3dbeam_results_output"; overwrite=true)
add_results_writer!(analysis, xdmf)

In [ ]:
add_problems!(analysis, beam, trac, bc)
run!(analysis)
close(xdmf)

# Thank you for your attention!